In [1]:
#!pip install ocs-academic-hub
#!pip uninstall -y ocs_academic_hub
#!pip list

In [2]:
#!git clone https://github.com/cfoisy-osisoft/academic-hub.git

In [3]:
#!jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e12

In [4]:
from ocs_academic_hub import OMFClient
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil.parser import parse
import time 
from time import sleep

C:\Users\zcheng\Anaconda3\lib\site-packages\typeguard\__init__.py:906: UserWarning: no type annotations present -- not typechecking ocs_academic_hub.omf_client.OMFClient.__init__
  warn('no type annotations present -- not typechecking {}'.format(function_name(func)))


# Data frame generation

In [5]:
start = pd.to_datetime('2021-04-01 15:47:00')
end = pd.to_datetime('2021-04-01 15:50:00')
freq = 20 #Hz
column = 4
batch_size = 50

In [6]:
time_range = pd.date_range(start, end, freq=f'{1/freq*1000000000}ns')

np.random.seed(seed=1111)
data = np.random.uniform(1, high=100, size=(len(time_range), column))

df = pd.DataFrame({'Timestamp': time_range}) 
#df = df.set_index('Timestamp')
i = 0
while i < column:
    df[f'col{i + 1}'] = data[:,i]
    i += 1
print(df)

                   Timestamp       col1       col2       col3       col4
0    2021-04-01 15:47:00.000  10.459371  92.575366  35.013769  31.737217
1    2021-04-01 15:47:00.050   1.198974  24.323878  24.541380  73.855671
2    2021-04-01 15:47:00.100  50.051340  78.658110  13.524125  61.058283
3    2021-04-01 15:47:00.150  47.145977  24.476080  44.080759  25.123480
4    2021-04-01 15:47:00.200  39.000151  84.000975  65.863288  15.696221
...                      ...        ...        ...        ...        ...
3596 2021-04-01 15:49:59.800  77.316408  98.087414  28.896870  71.278356
3597 2021-04-01 15:49:59.850  81.325958  57.351737  40.252543  24.371531
3598 2021-04-01 15:49:59.900   9.270223  86.376215  63.276650  40.157959
3599 2021-04-01 15:49:59.950  96.116558  35.603031  26.571837  67.697390
3600 2021-04-01 15:50:00.000  78.484158   6.750825  19.641894  31.234646

[3601 rows x 5 columns]


# Schedular 

In [7]:
from threading import Timer

class RepeatedTimer(object):
    def __init__(self, interval, function, *args, **kwargs):
        self._timer     = None
        self.interval   = interval
        self.function   = function
        self.args       = args
        self.kwargs     = kwargs
        self.is_running = False
        self.start()

    def _run(self):
        self.is_running = False
        self.start()
        self.function(*self.args, **self.kwargs)

    def start(self):
        if not self.is_running:
            self._timer = Timer(self.interval, self._run)
            self._timer.start()
            self.is_running = True

    def stop(self):
        self._timer.cancel()
        self.is_running = False

# Batching OMF messages to send to EDS

In [8]:
class Batch:
     def __init__(self):
          self._batch = pd.DataFrame(columns=df.columns)
       
     def get_batch(self):

         return self._batch

     def set_batch(self, a):
         if len(self._batch) < batch_size:
                self._batch = a
                #print(f'added to batch {self._batch}')
         elif len(self._batch) >= batch_size:
                #t = time.time()
                omf_client.update_tags(self._batch, asset)
                #print(f'sent the batch {self._batch}')
                self._batch = pd.DataFrame(columns=df.columns)
                #print(f'elapsed {time.time() - t}')
                #print(f'reset {self._batch}')
                
     def del_batch(self):
         del self._batch
     
     batch = property(get_batch, set_batch, del_batch) 

In [9]:
asset = 'test'
omf_client = OMFClient()
batch_df = Batch()

interval = 1 / freq

def update_batch(df_batch, df_rt, asset):
    batch_df.batch = df_batch.append(df_rt)
    
def event():
    df_rt = df[abs(df['Timestamp'] - pd.Timestamp.now()) <= timedelta(seconds = interval)]
    update_batch(batch_df.batch, df_rt, asset)
        
if pd.Timestamp.now() > start:
   print("starting...")
   rt = RepeatedTimer(1 / freq, event)  # it auto-starts, no need of rt.start()
   try:
       sleep((end - pd.Timestamp.now()).total_seconds())  # your long-running job goes here...
   finally:
       rt.stop()  # better in a try/finally block to make sure the program ends!
       print("stopped!!!")

starting...
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:50.800000 to 2021-04-01 15:48:52.400000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:50.800000 to 2021-04-01 15:48:52.400000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:50.800000 to 2021-04-01 15:48:52.400000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:50.800000 to 2021-04-01 15:48:52.400000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:50.800000 to 2021-04-01 15:48:52.400000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:50.800000 to 2021-04-01 15:48:52.400000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:50.800000 to 2021-04-01 15:48:52.400000
>

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s)

>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:48:55.250000 to 2021-04-01 15:48:56.750000
>> new tag(s)

...

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s)

>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s): ['x.test.col1', 'x.test.col2', 'x.test.col3', 'x.test.col4']
>> from 2021-04-01 15:49:21.400000 to 2021-04-01 15:49:23.050000
>> new tag(s)